In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb 
import catboost as ctb
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/msbd5001-fall2020/sampleSubmission.csv
/kaggle/input/msbd5001-fall2020/train.csv
/kaggle/input/msbd5001-fall2020/test.csv


In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import gc
from math import pi
from math import cos
from math import floor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from typing import Callable, List
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor, BayesianRidge, Lasso, HuberRegressor, ElasticNet, Lars
from sklearn.svm import LinearSVR, SVR, NuSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor, BaggingRegressor


class CosineAnnealingLearningRateSchedule(tf.keras.callbacks.Callback):
	# constructor
	def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
		self.epochs = n_epochs
		self.cycles = n_cycles
		self.lr_max = lrate_max
		self.lrates = list()
 
	# calculate learning rate for an epoch
	def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
		epochs_per_cycle = floor(n_epochs/n_cycles)
		cos_inner = (pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
		return lrate_max/2 * (cos(cos_inner) + 1)
 
	# calculate and set learning rate at the start of the epoch
	def on_epoch_begin(self, epoch, logs=None):
		# calculate learning rate
		lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
		# set learning rate
		tf.keras.backend.set_value(self.model.optimizer.lr, lr)
		# log value
		self.lrates.append(lr)

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [3]:
holidays =[
    "2017-1-1",
    "2017-1-2",
    "2017-1-28",
    "2017-1-30",
    "2017-1-31",
    "2017-4-4",
    "2017-4-14",
    "2017-4-15",
    "2017-4-17",
    "2017-5-1",
    "2017-5-3",
    "2017-5-30",
    "2017-7-1",
    "2017-10-1",
    "2017-10-2",
    "2017-10-5",
    "2017-10-28",
    "2017-12-25",
    "2017-12-26",
    "2018-1-1",
    "2018-2-16",
    "2018-2-17",
    "2018-2-19",
    "2018-3-30",
    "2018-3-31",
    "2018-4-2",
    "2018-4-5",
    "2018-5-1",
    "2018-5-22",
    "2018-6-18",
    "2018-7-1",
    "2018-7-2",
    "2018-9-24",
    "2018-9-25",
    "2018-10-1",
    "2018-10-17",
    "2018-12-25",
    "2018-12-26",
]
holidays = [pd.to_datetime(h, format='%Y-%m-%d').tz_localize("HongKong") for h in holidays]

In [4]:
from sklearn.model_selection import KFold, TimeSeriesSplit
random_state = 2020
# num_fold = 5

In [5]:
def read_files():
    sample_submission_df = pd.read_csv("/kaggle/input/msbd5001-fall2020/sampleSubmission.csv")
    sample_submission_df.shape

    train_df = pd.read_csv("/kaggle/input/msbd5001-fall2020/train.csv")
    test_df = pd.read_csv("/kaggle/input/msbd5001-fall2020/test.csv")
    train_df.date = pd.to_datetime(train_df.date, format='%d/%m/%Y %H:%M').dt.tz_localize("HongKong")
    test_df.date = pd.to_datetime(test_df.date, format='%d/%m/%Y %H:%M').dt.tz_localize("HongKong")
    
    test_df["is_test"] = True
    train_df["is_test"] = False

    all_df = pd.concat([train_df, test_df], ignore_index=True).sort_values("date").reset_index(drop=True)

    return sample_submission_df, train_df, test_df, all_df

In [6]:
def handle_missing_dates(train_df, test_df, all_df, hole_random_state):
    all_df["lag_date"] = all_df["date"].shift()
    all_df["interval"] = (all_df["date"] - all_df["lag_date"])
    train_df["lag_date"] = train_df["date"].shift()
    train_df["interval"] = (train_df["date"] - train_df["lag_date"])
    
    print("Missing intervals 2018")
    print(train_df[train_df.date.dt.year==2018].interval.value_counts())
    
    # all dates
    ideal_date = pd.period_range(
        start="2017-01-01 00:00:00",
        end="2018-12-31 23:00:00",
        freq="H")
    ideal_df = pd.DataFrame({
        "date": ideal_date.to_timestamp(),
    })
    ideal_df["date"] = ideal_df["date"].dt.tz_localize("HongKong")

    full_df = pd.merge(ideal_df, all_df, on="date", how="left")
    
    full_df["lag_date"] = full_df["date"].shift()
    full_df["interval"] = (full_df["date"] - full_df["lag_date"])
    full_df["interval"].value_counts()
    full_df = full_df[["date", "id", "speed", "is_test"]]
    
    
    full_df = full_df[~full_df.is_test.isna()]
    full_df["id"] = full_df["id"].astype(int)
    
    full_2017_df = full_df[full_df.date.dt.year==2017].reset_index(drop=True)
    full_2018_df = full_df[full_df.date.dt.year==2018].reset_index(drop=True)
    full_2017_df.shape, full_2018_df.shape
    
    date_holes_2018 = full_2018_df[full_2018_df.speed.isna()].date
    
    # simulate missing intervals in 2017 data
#     date_holes = full_2017_df.sample(frac=3504/full_2018_df.shape[0], random_state=hole_random_state).date
    date_holes = full_2018_df[full_2018_df.speed.isna()].date.apply(lambda x: x.replace(year=2017))
    
    print(date_holes_2018)
    print(date_holes)
    
    full_2017_df["truth"] = full_2017_df.speed
    full_2017_df.loc[full_2017_df.date.isin(date_holes), "speed"] = np.nan
    full_2017_df["speed"].isna().sum()
    
#     temp_df = full_2017_df[~full_2017_df.speed.isna()].reset_index(drop=True)
#     temp_df["lag_date"] = temp_df["date"].shift()
#     temp_df["interval"] = (temp_df["date"] - temp_df["lag_date"])
    
#     print("Mocking Missing intervals 2017")
#     print(temp_df[temp_df.date.dt.year == 2017]["interval"].value_counts())
    
    full_2018_df["truth"] = full_2018_df.speed
    full_df = pd.concat([full_2017_df, full_2018_df], ignore_index=True)
    return train_df, test_df, full_df, full_2017_df, full_2018_df

In [7]:
def speed_feature_engineering(df: pd.DataFrame, speed_feature="speed"):
    
    
    # % change
    for lag in [1, 2, 3]:
        df[f"Prev_{lag}_%_change"] = df["per_change"].shift() 
    
    # Prev/Post hours speed compareing to their weekday_hour average
    for lag in [1, 2, 3, 24]:
        df[f"Prev_{lag}_{speed_feature}"] = df[speed_feature].shift(lag)
        df[f"Post_{lag}_{speed_feature}"] = df[speed_feature].shift(-lag)
           
        df[f"Prev_{lag}_weekday_hour_mean"] = df["weekday_hour_avg_speed"].shift(lag)
        df[f"Post_{lag}_weekday_hour_mean"] = df["weekday_hour_avg_speed"].shift(-lag)
        
        df[f"Prev_{lag}_weekday_hour_std"] = df["weekday_hour_std_speed"].shift(lag)
        df[f"Post_{lag}_weekday_hour_std"] = df["weekday_hour_std_speed"].shift(-lag)
        
        
        # Prev/Post speed compare to Prev/Post mean spped
        df[f"Ratio_Prev_{lag}_weekday_hour_mean"] = df[f"Prev_{lag}_{speed_feature}"] / df[f"Prev_{lag}_weekday_hour_mean"]
        df[f"Ratio_Post_{lag}_weekday_hour_mean"] = df[f"Post_{lag}_{speed_feature}"] / df[f"Post_{lag}_weekday_hour_mean"]
        
        df[f"PDiff_Prev_{lag}_weekday_hour_mean"] = (df[f"Prev_{lag}_{speed_feature}"] - df[f"Prev_{lag}_weekday_hour_mean"]) / df[f"Prev_{lag}_weekday_hour_mean"]
        df[f"PDiff_Post_{lag}_weekday_hour_mean"] = (df[f"Post_{lag}_{speed_feature}"] - df[f"Post_{lag}_weekday_hour_mean"]) / df[f"Post_{lag}_weekday_hour_mean"]
        
        df[f"Z_Score_Prev_{lag}_weekday_hour"] = (df[f"Prev_{lag}_{speed_feature}"] - df[f"Prev_{lag}_weekday_hour_mean"]) / df[f"Prev_{lag}_weekday_hour_std"]
        df[f"Z_Score_Post_{lag}_weekday_hour"] = (df[f"Post_{lag}_{speed_feature}"] - df[f"Post_{lag}_weekday_hour_mean"]) / df[f"Post_{lag}_weekday_hour_std"]
    
    # % change between prevs & posts
    for lag in [3]:
        df[f"PDiff_Prev_1_{lag}"] = (df[f"Prev_{lag}_{speed_feature}"] - df[f"Prev_1_{speed_feature}"]) / df[f"Prev_{lag}_{speed_feature}"]
        

    # Prev/Post Moving simple statistics
    for win in [3, 5, 10]:
        df[f"Prev_{win}_Avg_{speed_feature}"] = df[speed_feature].shift().rolling(win, win_type=None, min_periods=1).mean()
        df[f"Prev_{win}_Max_{speed_feature}"] = df[speed_feature].shift().rolling(win, win_type=None, min_periods=1).max()
        df[f"Prev_{win}_Min_{speed_feature}"] = df[speed_feature].shift().rolling(win, win_type=None, min_periods=1).min()
        
        df[f"Post_{win}_Avg_{speed_feature}"] = df[f"Prev_{win}_Avg_{speed_feature}"].shift(-win-1)
        df[f"Post_{win}_Max_{speed_feature}"] = df[f"Prev_{win}_Max_{speed_feature}"].shift(-win-1)
        df[f"Post_{win}_Min_{speed_feature}"] = df[f"Prev_{win}_Min_{speed_feature}"].shift(-win-1)
    
        df[f"Ratio_Prev_2_{win}_Avg"] = df[f"Prev_2_{speed_feature}"] / (df[f"Prev_{win}_Avg_{speed_feature}"])
        
        df[f"Prev_{win}_EMA_{speed_feature}"] = df[speed_feature].shift().ewm(win, min_periods=1, ignore_na=True).mean()
    
    # Prev/Post Moving groupby statistics
#     for win in [3, 5]:
#         df[f"Prev_{win}_Avg_hourly_{speed_feature}"] = full_df.groupby("weekday_hour")[speed_feature].shift().rolling(win, min_periods=1).median()
# #         df[f"Post_{win}_Avg_hourly_{speed_feature}"] = df[f"Prev_{win}_Avg_hourly_{speed_feature}"].shift(-win-1)
    
    return df

def feature_engineering(full_df, fill_speed_func):
    full_df["month"] = full_df.date.dt.month
    full_df["day"] = full_df.date.dt.day
    full_df["hour"] = full_df.date.dt.hour # 24
    full_df["weekday"] = full_df.date.dt.weekday # 7
    full_df["dayofyear"] = full_df.date.dt.dayofyear
    full_df["weekofyear"] = full_df.date.dt.weekofyear

    full_df["holiday"] = full_df.date.dt.floor('d').isin(holidays)

    # sun -> 6 
    full_df.loc[full_df["weekday"]==0, "weekday"] = 7
    full_df["weekday"] -= 1

    full_df["weekday_hour"] = full_df["weekday"] * 24 + full_df["hour"]
    full_df["day_weekday_hour"] = (full_df["day"]//7) * (7*24) + full_df["weekday"] * 24 + full_df["hour"]
    full_df["quarter"] = full_df.date.dt.quarter
    full_df["year_weekday_hour"] = full_df.date.dt.year + full_df["weekday"] * 24 + full_df["hour"]

    full_df = full_df.sort_values("date").reset_index(drop=True)
    
    full_df["hour_avg_speed"] = full_df.groupby("hour")["truth"].transform("mean")
    full_df["hour_std_speed"] = full_df.groupby("hour")["truth"].transform("std")

    full_df["weekday_hour_avg_speed"] = full_df.groupby(["weekday_hour"])["truth"].transform("mean")
    full_df["weekday_hour_std_speed"] = full_df.groupby(["weekday_hour"])["truth"].transform("std")

    full_df["per_change"] = full_df["speed"].pct_change()
    
    full_df = fill_speed_func(full_df)

    return full_df

In [8]:

def prepare(full_df, fill_speed_func):

    
    full_df = feature_engineering(full_df, fill_speed_func)
    train_df = full_df[~full_df.speed.isna()].reset_index(drop=True)
    valid_df_2017 = full_df[(full_df.speed.isna()) & (full_df.date.dt.year==2017)].reset_index(drop=True)
    valid_df_2018 = full_df[(~full_df.speed.isna()) & (full_df.date.dt.year==2018)].reset_index(drop=True)
    test_df = full_df[(full_df.speed.isna()) & (full_df.date.dt.year==2018)].reset_index(drop=True)
    
    print(train_df.shape, valid_df_2017.shape, valid_df_2018.shape)
    
    selected_features = [col for col in train_df.columns if col not in [
        "id", "date", "per_change", "daily_max", "daily_min",
        "day_weekday_hour",
        "date_only",
        "year_weekday_hour",
        "speed_fill_median_h",
        "speed_fill_median_wh",
        "speed_fill_mean_h",
        "speed_fill_mean_wh",
        "speed_fill_mean",
        "speed_fill_median", 
        "speed_fill_wh_median",
        "lin_speed", "quadratic_speed", "nearest_speed", "speed", "is_test", "truth"
    ]]
    
    # fill nans
    for col in selected_features:
        avg = train_df[col].mean()
        for df in [train_df, test_df, valid_df_2017, valid_df_2018]:
            df[col] = df[col].fillna(avg)
    
    return train_df, valid_df_2017, valid_df_2018, test_df, selected_features


In [9]:

def train(model_builder: Callable,
          train_df, 
          valid_df_2017,
          valid_df_2018,
          test_df,
          features: List[str],
          scaling=True,
          one_hot_features=[]):
    all_train_preds = np.array([])
    all_train_y = np.array([])
    
    all_valid_preds_2017 = np.array([])
    all_valid_y_2017 = np.array([])
    all_valid_preds_2018 = np.array([])
    all_valid_y_2018 = np.array([])
    
    all_test_preds = 0
    
    models = []
    num_fold = 1
    feature_importance = None
    
    for fold in range(num_fold):
        _train_df = train_df.copy()
        _valid_df_2017 = valid_df_2017.copy()
        _valid_df_2018 = valid_df_2018.copy()
        
        _features = features.copy()
        _features = [f for f in _features if f not in one_hot_features]
        
        train_ohe = {}
        valid_2017_ohe = {}
        valid_2018_ohe = {}
        test_ohe = {}
        for f in one_hot_features:
            labeler = LabelEncoder()
            labeler.fit(_train_df[f])
            
            train_ohe[f] = tf.keras.utils.to_categorical(labeler.transform(_train_df[f]))
            valid_2017_ohe[f] = tf.keras.utils.to_categorical(labeler.transform(_valid_df_2017[f]))
            valid_2018_ohe[f] = tf.keras.utils.to_categorical(labeler.transform(_valid_df_2018[f]))
            test_ohe[f] = tf.keras.utils.to_categorical(labeler.transform(test_df[f]))
            
        _train_x = _train_df[_features].values
        _valid_x_2017 = _valid_df_2017[_features].values
        _valid_x_2018 = _valid_df_2018[_features].values
        _test_x = test_df[_features].copy().values
        
        if scaling:
            scaler = StandardScaler()
            scaler.fit(_train_x)
            
            _train_x = scaler.transform(_train_x)
            _valid_x_2017 = scaler.transform(_valid_x_2017)
            _valid_x_2018 = scaler.transform(_valid_x_2018)
            _test_x = scaler.transform(_test_x)
        
        for ohe_f in train_ohe.keys():
            _train_x = np.concatenate([_train_x, train_ohe[ohe_f]], axis=1)
            _valid_x_2017 = np.concatenate([_valid_x_2017, valid_2017_ohe[ohe_f]], axis=1)
            _valid_x_2018 = np.concatenate([_valid_x_2018, valid_2018_ohe[ohe_f]], axis=1)
            _test_x = np.concatenate([_test_x, test_ohe[ohe_f]], axis=1)
        
        _train_y = _train_df.speed.values
        _valid_y_2017 = _valid_df_2017.truth.values
        _valid_y_2018 = _valid_df_2018.truth.values            
        
        model = model_builder()
        num_seed = 5
        if isinstance(model, tf.keras.Model):
            for seed in range(0, num_seed):
                model = model_builder()
                model.compile("adam", loss="mse")
                history = model.fit(
                    _train_x,
                    _train_y,
                    validation_data=(
                        _valid_x_2017, _valid_y_2017
                    ),
                    batch_size=200, epochs=100,
                    verbose=0,
                    callbacks=[
                        CosineAnnealingLearningRateSchedule(100, 10, 0.001),
                        tf.keras.callbacks.ModelCheckpoint(f'model_{fold}_{seed}.h5', save_best_only=True, verbose=0, monitor='val_loss'),
                    ]
                )
                print(min(history.history["val_loss"]))
        elif isinstance(model, lgb.LGBMRegressor):
            sample_weights = [1 if 20 <= h <= 23 else 1 for h in train_df.hour]
            model.fit(
                _train_x,
                _train_y,
                eval_set=(
                    _valid_x_2017, _valid_y_2017
                ),
                early_stopping_rounds=100,
                verbose=False,
                sample_weight=sample_weights,
            )
            
            feature_importance = model.feature_importances_
        elif isinstance(model, ctb.CatBoostRegressor):
            model.fit(
                _train_x,
                _train_y,
                eval_set=(
                    _valid_x_2017, _valid_y_2017
                ),
                early_stopping_rounds=100,
                verbose=False,
            ) 
        else:
            model.fit(
                _train_x,
                _train_y,
            )
        models.append(model)
        
        train_preds = 0
        valid_preds_2017 = 0
        valid_preds_2018 = 0
        test_preds = 0       
        if isinstance(model, tf.keras.Model):
            for seed in range(num_seed):
                model.load_weights(f'model_{fold}_{seed}.h5')
                train_preds += model.predict(_train_x).reshape(-1) / num_seed
                valid_preds_2017 += model.predict(_valid_x_2017).reshape(-1) / num_seed
                valid_preds_2018 += model.predict(_valid_x_2018).reshape(-1) / num_seed
                test_preds += model.predict(_test_x).reshape(-1) / num_seed
        
        else:
            train_preds = model.predict(_train_x).reshape(-1)
            valid_preds_2017 = model.predict(_valid_x_2017).reshape(-1)
            valid_preds_2018 = model.predict(_valid_x_2018).reshape(-1)
            test_preds = model.predict(_test_x).reshape(-1)
        all_test_preds += test_preds / num_fold


        train_mse = mean_squared_error(_train_y, train_preds)
        valid_mse_2017 = mean_squared_error(_valid_y_2017, valid_preds_2017)
        valid_mse_2018 = mean_squared_error(_valid_y_2018, valid_preds_2018)
#         print(train_mse, valid_mse_2017, valid_mse_2018)

#         print(train_preds.shape)
        all_train_preds = np.concatenate([all_train_preds, train_preds]) 
        all_train_y = np.concatenate([all_train_y, _train_y])

        all_valid_preds_2017 = np.concatenate([all_valid_preds_2017, valid_preds_2017]) 
        all_valid_y_2017 = np.concatenate([all_valid_y_2017, _valid_y_2017]) 

        all_valid_preds_2018 = np.concatenate([all_valid_preds_2018, valid_preds_2018]) 
        all_valid_y_2018 = np.concatenate([all_valid_y_2018, _valid_y_2018]) 
    
    train_mse = mean_squared_error(all_train_y, all_train_preds)
    valid_mse_2017 = mean_squared_error(all_valid_y_2017, all_valid_preds_2017)
    valid_mse_2018 = mean_squared_error(all_valid_y_2018, all_valid_preds_2018)
    
    print(train_mse, valid_mse_2017, valid_mse_2018)
    return {
        "models": models,
        "all_train_preds": all_train_preds,
        "all_train_y": all_train_y,
        
        "all_valid_preds_2017": all_valid_preds_2017,
        "all_valid_y_2017": all_valid_y_2017,
        "all_valid_preds_2018": all_valid_preds_2018,
        "all_valid_y_2018": all_valid_y_2018,
        "test_preds": all_test_preds,
        "features": _features,
        "feature_importance": feature_importance, 
        "valid_mse_2017": valid_mse_2017,
    }

In [10]:
def train_models(train_df, valid_df_2017, valid_df_2018, test_df, selected_features):
    init = tf.keras.initializers.LecunNormal()
    keras_one_hot_features = ["hour", "weekday"]
    
    model_builders = {
        "lgb": lambda :lgb.LGBMRegressor(
                boosting_type='gbdt', 
                objective="mse",
                min_child_samples=20,
                num_leaves=40,
                max_depth=16,
                learning_rate=0.05,
                n_estimators=1000,
                colsample_bytree=.4,
                subsample=1.0,
                subsample_freq=0,
                importance_type="gain",
                random_state=random_state,
                n_jobs=-1,
        ),
        'lgbrf': lambda :lgb.LGBMRegressor(
                boosting_type='rf', 
                objective="mse",
                min_child_samples=20,
                num_leaves=128,
                max_depth=16,
                learning_rate=0.05,
                n_estimators=1000,
                colsample_bytree=.5,
                subsample=.95,
                subsample_freq=20,
                importance_type="gain",
                random_state=random_state,
                n_jobs=-1,
        ),
        'cat': lambda :ctb.CatBoostRegressor(
                loss_function="RMSE",
                learning_rate=.06,
                max_depth=8,
                min_child_samples=20,
                colsample_bylevel=1.0,
                n_estimators=None,
                use_best_model=True,
                random_seed=random_state,
        ),
        'keras': lambda :tf.keras.Sequential(
                    layers=[
                        tf.keras.Input(shape=(len(selected_features) - len(keras_one_hot_features) + sum([full_df[f].nunique() for f in keras_one_hot_features]),)),  

                        tf.keras.layers.Dense(20, kernel_initializer=init, activation=tfa.activations.mish),
                        tf.keras.layers.Dropout(.05),
                        tf.keras.layers.BatchNormalization(),
                        tf.keras.layers.GaussianNoise(.15),

                        tf.keras.layers.Dense(20, kernel_initializer=init, activation="tanh"),
                        tf.keras.layers.Dropout(.05),
                        tf.keras.layers.BatchNormalization(),

                        tf.keras.layers.Dense(1, kernel_initializer=init, activation="linear"),
                    ]
        ),
        'lr': lambda :LinearRegression(),
        'rr': lambda :Ridge(),
        'sdg': lambda :SGDRegressor(max_iter=800),
        'etr': lambda :ExtraTreesRegressor(
                    n_estimators=400,
                    max_depth=18,
                    max_leaf_nodes=None,
                    criterion="mse",
                    min_samples_split=25,
                    max_features=.9,
                    bootstrap=False,
                    random_state=random_state,
                    n_jobs=-1,
        ),
        'hgbr': lambda :HistGradientBoostingRegressor(
                    max_iter=200,
                    max_leaf_nodes=64,
                    max_depth=16,
                    learning_rate=.04,
                    random_state=random_state,
        ),
        'svr': lambda :SVR(C=300),
        'mlp': lambda :
                MLPRegressor(
                    hidden_layer_sizes=(16, 16), activation='tanh', solver='adam', 
                    learning_rate="adaptive", shuffle=True,
                    max_iter=120, random_state=random_state,
        ),
        'gbr': lambda :GradientBoostingRegressor(
            subsample=.8,
            n_estimators=120,
            min_samples_leaf=30, 
            max_depth=8,
            learning_rate=.04,
            criterion="mse",
            min_samples_split=20,
            random_state=random_state,
        ),
        'rf': lambda :RandomForestRegressor(
            n_estimators=400,
            max_depth=12,
            min_samples_leaf=6,
            max_features=.4,
            random_state=random_state,
            n_jobs=-1,
        ),
        'knn': lambda :KNeighborsRegressor(
            n_neighbors=20,
            weights="distance",
            n_jobs=-1,
        )
    }
    
    print("#"*5, "Models", "#"*5)
    model_results = {}
    for key, model_builder in model_builders.items():
        scaling = False
        one_hot_features = []
        
        if key == 'keras':
            one_hot_features = keras_one_hot_features
        
        if key in ['keras', 'lr', 'rr', 'sdg', 'mlp', 'rf', 'knn']:
            scaling = True
        
        print(key)
        result = train(
            model_builder,
            train_df,
            valid_df_2017,
            valid_df_2018,
            test_df,
            selected_features,
            scaling=scaling,
            one_hot_features=one_hot_features,
        )
        model_results[key] = result
    return model_results
    

In [11]:
def blending(model_results):
    train_preds_df = pd.DataFrame({
        k: r["all_train_preds"] for k, r in model_results.items()
    })
    train_preds_df["y"] = model_results[list(model_results.keys())[0]]["all_train_y"]
    
    
    oof_df_2017 = pd.DataFrame({
        k: r["all_valid_preds_2017"] for k, r in model_results.items()
    })
    oof_df_2017["y"] = model_results[list(model_results.keys())[0]]["all_valid_y_2017"]

    
    oof_cols = [col for col in oof_df_2017.columns if col not in ["y"]]
    
    blending_lr = LinearRegression()

    # train blender with 2017 data
    blending_lr.fit(
        oof_df_2017[oof_cols].values,
        oof_df_2017["y"].values,
    )
    

#     display(pd.DataFrame({
#         "model": oof_cols,
#         "weight": blending_lr.coef_
#     }).sort_values("weight"))

    # blend valid
    oof_df_2017["blended"] =  0
    for i in range(len(oof_cols)):
        oof_df_2017["blended"] += oof_df_2017[oof_cols[i]] * blending_lr.coef_[i]
    oof_df_2017["blended"] += blending_lr.intercept_
    
    # blend train
    train_preds_df["blended"] =  0
    for i in range(len(oof_cols)):
        train_preds_df["blended"] += train_preds_df[oof_cols[i]] * blending_lr.coef_[i]
    train_preds_df["blended"] += blending_lr.intercept_

    
    print("Blending CV")
    mse_2017 = mean_squared_error(oof_df_2017["y"], oof_df_2017["blended"])
    print(mean_squared_error(train_preds_df["y"], train_preds_df["blended"]), mse_2017)
    
    # blend submissions
    sub_df = pd.DataFrame({
        k: r["test_preds"] for k, r in model_results.items()
    })
    
    sub_df["blended"] =  0
    for i in range(len(oof_cols)):
        sub_df["blended"] += sub_df[oof_cols[i]] * blending_lr.coef_[i]
    sub_df["blended"] += blending_lr.intercept_

    test_df["speed"] = sub_df["blended"].values
#     test_df[["id", "speed"]].to_csv("submission.csv", index=False)

    return train_preds_df, oof_df_2017, test_df[["id", "speed"]], mse_2017

In [12]:
import warnings
warnings.filterwarnings('ignore')

# Stage 1

In [13]:
def fill_speed_func(df):
    df["speed_fill_median"] = df.groupby("weekday_hour")["speed"].transform("median")
    df.loc[~full_df.speed.isna(), "speed_fill_median"] = df.loc[~df.speed.isna(), "speed"]
    df = speed_feature_engineering(df, 'speed_fill_median')
    return df

subs = []
sample_submission_df, train_df, test_df, all_df = read_files()
train_df, test_df, full_df, full_2017_df, full_2018_df = handle_missing_dates(
    train_df, test_df, all_df, hole_random_state=2020)
train_df, valid_df_2017, valid_df_2018, test_df, selected_features = prepare(full_df, fill_speed_func)
model_results = train_models(train_df, valid_df_2017, valid_df_2018, test_df, selected_features)

blended_train_preds, blended_val_preds, blended_sub, mse_2017 = blending(model_results)
blended_sub[["id", "speed"]].to_csv(f"submission_{i}.csv", index=False)

mse_2017

Missing intervals 2018
0 days 01:00:00    3135
0 days 02:00:00    1260
0 days 03:00:00     526
0 days 04:00:00     217
0 days 05:00:00      75
0 days 06:00:00      30
0 days 07:00:00       8
0 days 10:00:00       2
0 days 08:00:00       2
0 days 09:00:00       1
Name: interval, dtype: int64
2      2018-01-01 02:00:00+08:00
5      2018-01-01 05:00:00+08:00
7      2018-01-01 07:00:00+08:00
8      2018-01-01 08:00:00+08:00
10     2018-01-01 10:00:00+08:00
                  ...           
8753   2018-12-31 17:00:00+08:00
8755   2018-12-31 19:00:00+08:00
8757   2018-12-31 21:00:00+08:00
8758   2018-12-31 22:00:00+08:00
8759   2018-12-31 23:00:00+08:00
Name: date, Length: 3504, dtype: datetime64[ns, Hongkong]
2      2017-01-01 02:00:00+08:00
5      2017-01-01 05:00:00+08:00
7      2017-01-01 07:00:00+08:00
8      2017-01-01 08:00:00+08:00
10     2017-01-01 10:00:00+08:00
                  ...           
8753   2017-12-31 17:00:00+08:00
8755   2017-12-31 19:00:00+08:00
8757   2017-12-31 21:00

NameError: name 'i' is not defined

In [14]:
stage_1_model_results = model_results

In [15]:
selected_features

['month',
 'day',
 'hour',
 'weekday',
 'dayofyear',
 'weekofyear',
 'holiday',
 'weekday_hour',
 'quarter',
 'hour_avg_speed',
 'hour_std_speed',
 'weekday_hour_avg_speed',
 'weekday_hour_std_speed',
 'Prev_1_%_change',
 'Prev_2_%_change',
 'Prev_3_%_change',
 'Prev_1_speed_fill_median',
 'Post_1_speed_fill_median',
 'Prev_1_weekday_hour_mean',
 'Post_1_weekday_hour_mean',
 'Prev_1_weekday_hour_std',
 'Post_1_weekday_hour_std',
 'Ratio_Prev_1_weekday_hour_mean',
 'Ratio_Post_1_weekday_hour_mean',
 'PDiff_Prev_1_weekday_hour_mean',
 'PDiff_Post_1_weekday_hour_mean',
 'Z_Score_Prev_1_weekday_hour',
 'Z_Score_Post_1_weekday_hour',
 'Prev_2_speed_fill_median',
 'Post_2_speed_fill_median',
 'Prev_2_weekday_hour_mean',
 'Post_2_weekday_hour_mean',
 'Prev_2_weekday_hour_std',
 'Post_2_weekday_hour_std',
 'Ratio_Prev_2_weekday_hour_mean',
 'Ratio_Post_2_weekday_hour_mean',
 'PDiff_Prev_2_weekday_hour_mean',
 'PDiff_Post_2_weekday_hour_mean',
 'Z_Score_Prev_2_weekday_hour',
 'Z_Score_Post_2_we

# Stage 2

In [16]:
def fill_speed_func(df):
    df["speed_fill_median"] = df["speed"]
    # 2017 holes
    df.loc[(df.date.dt.year==2017)&(df.speed.isna()), "speed_fill_median"] = blended_val_preds["blended"].values
    # 2018 (test holes)
    df.loc[(df.date.dt.year==2018)&(df.speed.isna()), "speed_fill_median"] = blended_sub["speed"].values

    df = speed_feature_engineering(df, 'speed_fill_median')
    return df

subs = []
sample_submission_df, train_df, test_df, all_df = read_files()
train_df, test_df, full_df, full_2017_df, full_2018_df = handle_missing_dates(
    train_df, test_df, all_df, hole_random_state=2020)
train_df, valid_df_2017, valid_df_2018, test_df, selected_features = prepare(full_df, fill_speed_func)
model_results = train_models(train_df, valid_df_2017, valid_df_2018, test_df, selected_features)

blended_train_preds, blended_val_preds, blended_sub, mse_2017 = blending(model_results)
blended_sub[["id", "speed"]].to_csv(f"submission_{i}.csv", index=False)

mse_2017

Missing intervals 2018
0 days 01:00:00    3135
0 days 02:00:00    1260
0 days 03:00:00     526
0 days 04:00:00     217
0 days 05:00:00      75
0 days 06:00:00      30
0 days 07:00:00       8
0 days 10:00:00       2
0 days 08:00:00       2
0 days 09:00:00       1
Name: interval, dtype: int64
2      2018-01-01 02:00:00+08:00
5      2018-01-01 05:00:00+08:00
7      2018-01-01 07:00:00+08:00
8      2018-01-01 08:00:00+08:00
10     2018-01-01 10:00:00+08:00
                  ...           
8753   2018-12-31 17:00:00+08:00
8755   2018-12-31 19:00:00+08:00
8757   2018-12-31 21:00:00+08:00
8758   2018-12-31 22:00:00+08:00
8759   2018-12-31 23:00:00+08:00
Name: date, Length: 3504, dtype: datetime64[ns, Hongkong]
2      2017-01-01 02:00:00+08:00
5      2017-01-01 05:00:00+08:00
7      2017-01-01 07:00:00+08:00
8      2017-01-01 08:00:00+08:00
10     2017-01-01 10:00:00+08:00
                  ...           
8753   2017-12-31 17:00:00+08:00
8755   2017-12-31 19:00:00+08:00
8757   2017-12-31 21:00

NameError: name 'i' is not defined

In [17]:
stage_2_model_results = model_results

In [18]:
selected_features

['month',
 'day',
 'hour',
 'weekday',
 'dayofyear',
 'weekofyear',
 'holiday',
 'weekday_hour',
 'quarter',
 'hour_avg_speed',
 'hour_std_speed',
 'weekday_hour_avg_speed',
 'weekday_hour_std_speed',
 'Prev_1_%_change',
 'Prev_2_%_change',
 'Prev_3_%_change',
 'Prev_1_speed_fill_median',
 'Post_1_speed_fill_median',
 'Prev_1_weekday_hour_mean',
 'Post_1_weekday_hour_mean',
 'Prev_1_weekday_hour_std',
 'Post_1_weekday_hour_std',
 'Ratio_Prev_1_weekday_hour_mean',
 'Ratio_Post_1_weekday_hour_mean',
 'PDiff_Prev_1_weekday_hour_mean',
 'PDiff_Post_1_weekday_hour_mean',
 'Z_Score_Prev_1_weekday_hour',
 'Z_Score_Post_1_weekday_hour',
 'Prev_2_speed_fill_median',
 'Post_2_speed_fill_median',
 'Prev_2_weekday_hour_mean',
 'Post_2_weekday_hour_mean',
 'Prev_2_weekday_hour_std',
 'Post_2_weekday_hour_std',
 'Ratio_Prev_2_weekday_hour_mean',
 'Ratio_Post_2_weekday_hour_mean',
 'PDiff_Prev_2_weekday_hour_mean',
 'PDiff_Post_2_weekday_hour_mean',
 'Z_Score_Prev_2_weekday_hour',
 'Z_Score_Post_2_we

# Combine

In [19]:
combine_model_results = {}
for key, item in stage_1_model_results.items():
    combine_model_results[f"{key}_1"] = item
for key, item in stage_2_model_results.items():
    combine_model_results[f"{key}_2"] = item
combine_model_results.keys()

dict_keys(['lgb_1', 'lgbrf_1', 'cat_1', 'keras_1', 'lr_1', 'rr_1', 'sdg_1', 'etr_1', 'hgbr_1', 'svr_1', 'mlp_1', 'gbr_1', 'rf_1', 'knn_1', 'lgb_2', 'lgbrf_2', 'cat_2', 'keras_2', 'lr_2', 'rr_2', 'sdg_2', 'etr_2', 'hgbr_2', 'svr_2', 'mlp_2', 'gbr_2', 'rf_2', 'knn_2'])

In [20]:
blended_train_preds, blended_val_preds, blended_sub, mse_2017 = blending(combine_model_results)
blended_sub[["id", "speed"]].to_csv(f"submission.csv", index=False)

Blending CV
5.115833163149732 8.836303493417866
